# AR6 calibration of FaIR 2.1

- use the exact aerosol indirect forcing relationship from Smith et al. (2021). 
- aerosol direct is from AR6.
- three layer model for climate response.
- include overlap of the major GHGs.
- prognostic equation for land use related forcing (e.g. from FaIR 1.6).
- ozone relationship from FaIR 1.6 used in AR6.
- interactive methane lifetime (NEW!)
- rough hack for RCP scenarios

We have to do this slightly differently to the examples so far. 1.5 million ensemble members is going to take up too much memory, so we run in batches of 1000, initialising a new FaIR instance for each batch, and saving the output as we go.

## Basic imports

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import scipy.interpolate
import scipy.stats
from tqdm.auto import tqdm
import xarray as xr

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
from fair.forcing.ghg import meinshausen2020

In [ ]:
os.makedirs('../data/ar6_ensemble_batches_rcp/', exist_ok=True)

## Set up problem

In [ ]:
erf_2co2 = meinshausen2020(
    np.array([554.30, 731.41, 273.87]) * np.ones((1, 1, 1, 3)),
    np.array([277.15, 731.41, 273.87]) * np.ones((1, 1, 1, 3)),
    np.array((1.05, 0.86, 1.07)) * np.ones((1, 1, 1, 1)),
    np.ones((1, 1, 1, 3)),
    np.array([True, False, False]),
    np.array([False, True, False]),
    np.array([False, False, True]),
    np.array([False, False, False])
).squeeze()[0]
erf_2co2

In [ ]:
scenarios = ['rcp45']

In [ ]:
df_forc = pd.read_csv('../data/forcing/table_A3.3_historical_ERF_1750-2019_best_estimate.csv')
df_solar = pd.read_csv('../data/forcing/solar_erf.csv', index_col='year')

In [ ]:
solar_forcing = df_solar['solar_erf'].loc[1765.5:2101].values
volcanic_forcing = np.zeros(336)
volcanic_forcing[:255] = df_forc['volcanic'].values[15:]
volcanic_forcing[254:266] = np.linspace(1, 0, 12) * volcanic_forcing[254]

In [ ]:
da_emissions = xr.load_dataarray('../data/rcmip/rcp_emissions_fair2.1.nc')

In [ ]:
da_emissions

In [ ]:
species = [
    'CO2 FFI', 'CO2 AFOLU', 'CO2', 'CH4', 'N2O',
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO',
    'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115',
    'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CH3Cl', 'CH3CCl3', 'CH3Br',
    'Halon-1211', 'Halon-1202', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C6F14',
    'SF6',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-227ea', 'HFC-23', 'HFC-245fa', 'HFC-32', 
    'HFC-4310mee', 'Solar', 'Volcanic', 'Aerosol-radiation interactions',
    'Aerosol-cloud interactions', 'Ozone', 'Light absorbing particles on snow and ice',
    'Land use', 'Stratospheric water vapour', 'Equivalent effective stratospheric chlorine',
    'Contrails', 'NOx aviation'
]

species, properties = read_properties(species=species)

In [ ]:
df_cc=pd.read_csv('../data/parameter_sets/carbon_cycle.csv')
df_cr=pd.read_csv('../data/parameter_sets/climate_response.csv')
df_aci=pd.read_csv('../data/parameter_sets/erfaci.csv')
df_ari=pd.read_csv('../data/parameter_sets/erfari.csv')
df_ozone=pd.read_csv('../data/parameter_sets/ozone.csv')
df_scaling=pd.read_csv('../data/parameter_sets/forcing_scaling.csv')
df_1750co2=pd.read_csv('../data/parameter_sets/co2_concentration_1750.csv')

In [ ]:
# using an extremely crude curve fit to estimate 1765 CO2

obs_co2 = np.array([278.3, 285.5, 296.4])
obs_years = [1750, 1850, 1900]

def fit(x, a, b, c):
    return a + b*np.exp(c*(x-1750))

p, _ = scipy.optimize.curve_fit(fit, obs_years, obs_co2, p0=[278.3, 1.45, 0.0171])
pl.plot(np.arange(1750, 1900), p[0]+p[1]*np.exp(p[2] * (np.arange(1750, 1900)-1750)))
pl.scatter(obs_years, obs_co2, color='k')

offset1765co2 = fit(1765, p[0], p[1], p[2]) - fit(1750, p[0], p[1], p[2])
offset1765co2

In [ ]:
df_1765co2 = df_1750co2 + offset1765co2

In [ ]:
obs_ch4 = np.array([729.2, 807.6, 925.1])
obs_years = [1750, 1850, 1900]

def fit(x, a, b, c):
    return a + b*np.exp(c*(x-1750))

p, _ = scipy.optimize.curve_fit(fit, obs_years, obs_ch4, p0=[278.3, 1.45, 0.0171])
pl.plot(np.arange(1750, 1900), p[0]+p[1]*np.exp(p[2] * (np.arange(1750, 1900)-1750)))
pl.scatter(obs_years, obs_ch4, color='k')

baseline1765ch4 = fit(1765, p[0], p[1], p[2])
baseline1765ch4

In [ ]:
obs_n2o = np.array([270.1, 272.1, 278.9])
obs_years = [1750, 1850, 1900]

def fit(x, a, b, c):
    return a + b*np.exp(c*(x-1750))

p, _ = scipy.optimize.curve_fit(fit, obs_years, obs_n2o, p0=[278.3, 1.45, 0.0171])
pl.plot(np.arange(1750, 1900), p[0]+p[1]*np.exp(p[2] * (np.arange(1750, 1900)-1750)))
pl.scatter(obs_years, obs_n2o, color='k')

baseline1765n2o = fit(1765, p[0], p[1], p[2])
baseline1765n2o

## Generate 1.5 million ensemble members in batches of 1000

In [ ]:
samples = 1500000
batch_size = 1000

In [ ]:
species[5:12]

In [ ]:
seedgen = 1355763
seedstep = 399
#run_config = RunConfig(ch4_lifetime_method=CH4LifetimeMethod.AERCHEMMIP)

trend_shape = np.ones(336)
trend_shape[:256] = np.linspace(0, 1, 256)

# for all except temperature, the full time series is not important so we can save a bit of space
temp_out = np.ones((251, samples)) * np.nan
ohc_out = np.ones((samples)) * np.nan
fari_out = np.ones((samples)) * np.nan
faci_out = np.ones((samples)) * np.nan
co2_out = np.ones((samples)) * np.nan
fo3_out = np.ones((samples)) * np.nan
ecs = np.ones(samples) * np.nan
tcr = np.ones(samples) * np.nan

calibrated_f4co2_mean = df_cr['F_4xCO2'].mean()

for ibatch, batch_start in tqdm(enumerate(range(0, samples, batch_size)), total=samples/batch_size):
    batch_end = batch_start + batch_size
    
    f = FAIR(ch4_method='Thornhill2021')
    f.define_time(1765, 2100, 1)
    f.define_scenarios(scenarios)
    f.define_configs(list(range(batch_start, batch_end)))
    f.define_species(species, properties)
    f.allocate()
    
    # emissions and forcing
    #f.fill_from_rcmip()
    da = da_emissions.loc[dict(config='unspecified', scenario='rcp45')][:335, ...]
    fe = da.expand_dims(dim=['scenario', 'config'], axis=(1,2))
    f.emissions = fe.drop('config') * np.ones((1,1,batch_size,1))
    f.emissions[:86,0,:,5] = np.linspace(2, f.emissions[85,0,0,5], 86)[:, None]
    f.emissions[:86,0,:,6] = np.linspace(1.2, f.emissions[85,0,0,6], 86)[:, None]
    f.emissions[:86,0,:,7] = np.linspace(10, f.emissions[85,0,0,7], 86)[:, None]
    f.emissions[:86,0,:,8] = np.linspace(4, f.emissions[85,0,0,8], 86)[:, None]
    f.emissions[:86,0,:,9] = np.linspace(46/14*2, f.emissions[85,0,0,9], 86)[:, None]
    f.emissions[:86,0,:,10] = np.linspace(10, f.emissions[85,0,0,10], 86)[:, None]
    f.emissions[:86,0,:,11] = np.linspace(174, f.emissions[85,0,0,11], 86)[:, None]      
    
    fill(f.forcing, volcanic_forcing[:, None, None] * df_scaling.loc[batch_start:batch_end-1, 'Volcanic'].values.squeeze(), specie='Volcanic')
    fill(f.forcing, 
         solar_forcing[:, None, None] * 
         df_scaling.loc[batch_start:batch_end-1, 'solar_amplitude'].values.squeeze() + 
         trend_shape[:, None, None] * df_scaling.loc[batch_start:batch_end-1, 'solar_trend'].values.squeeze(),
         specie='Solar'
    )
    
    # climate response
    fill(f.climate_configs['ocean_heat_capacity'], df_cr.loc[batch_start:batch_end-1, 'c1':'c3'].values)
    fill(f.climate_configs['ocean_heat_transfer'], df_cr.loc[batch_start:batch_end-1, 'kappa1':'kappa3'].values)
    fill(f.climate_configs['deep_ocean_efficacy'], df_cr.loc[batch_start:batch_end-1, 'epsilon'].values.squeeze())
    fill(f.climate_configs['gamma_autocorrelation'], df_cr.loc[batch_start:batch_end-1, 'gamma'].values.squeeze())
    fill(f.climate_configs['sigma_eta'], df_cr.loc[batch_start:batch_end-1, 'sigma_eta'].values.squeeze())
    fill(f.climate_configs['sigma_xi'], df_cr.loc[batch_start:batch_end-1, 'sigma_xi'].values.squeeze())
    fill(f.climate_configs['seed'], np.arange(seedgen+batch_start*seedstep, seedgen+batch_end*seedstep, seedstep, dtype=int))
    fill(f.climate_configs['stochastic_run'], True)
    fill(f.climate_configs['use_seed'], True)
    fill(f.climate_configs['forcing_4co2'], 2 * erf_2co2 * (1 + 0.561*(calibrated_f4co2_mean - df_cr.loc[batch_start:batch_end-1,'F_4xCO2'])/calibrated_f4co2_mean))
    
    # species level
    f.fill_species_configs()
    
    # carbon cycle
    fill(f.species_configs['iirf_0'], df_cc.loc[batch_start:batch_end-1, 'r0'].values.squeeze(), specie='CO2')
    fill(f.species_configs['iirf_airborne'], df_cc.loc[batch_start:batch_end-1, 'rA'].values.squeeze(), specie='CO2')
    fill(f.species_configs['iirf_uptake'], df_cc.loc[batch_start:batch_end-1, 'rU'].values.squeeze(), specie='CO2')
    fill(f.species_configs['iirf_temperature'], df_cc.loc[batch_start:batch_end-1, 'rT'].values.squeeze(), specie='CO2')
    
    # aerosol indirect
    fill(f.species_configs['aci_scale'], df_aci.loc[batch_start:batch_end-1, 'beta'].values.squeeze())
    fill(f.species_configs['aci_shape'], df_aci.loc[batch_start:batch_end-1, 'shape_so2'].values.squeeze(), specie='Sulfur')
    fill(f.species_configs['aci_shape'], df_aci.loc[batch_start:batch_end-1, 'shape_bc'].values.squeeze(), specie='BC')
    fill(f.species_configs['aci_shape'], df_aci.loc[batch_start:batch_end-1, 'shape_oc'].values.squeeze(), specie='OC')
        
    # methane lifetime baseline
    fill(f.species_configs['unperturbed_lifetime'], 10.4198121, specie='CH4')
    
    # emissions adjustments for N2O and CH4 (we don't want to make these defaults as people might wanna run pulse expts with these gases)
    fill(f.species_configs['baseline_emissions'], 19.019783117809567, specie='CH4')
    fill(f.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')
    
    # aerosol direct
    for specie in df_ari:
        fill(f.species_configs['erfari_radiative_efficiency'], df_ari.loc[batch_start:batch_end-1, specie], specie=specie)
    
    # forcing
    for specie in df_scaling:
        if specie in ['minorGHG', 'solar_amplitude', 'solar_trend', 'CO2', 'Volcanic']:
            continue
        fill(f.species_configs['forcing_scale'], df_scaling.loc[batch_start:batch_end-1, specie].values.squeeze(), specie=specie)
    for specie in ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
        'CCl4', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1202', 'Halon-1211', 'Halon-1301', 'Halon-2402',
        'CF4', 'C2F6', 'C6F14', 'SF6',
        'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-227ea', 'HFC-23', 'HFC-245fa', 'HFC-32', 'HFC-4310mee']:
        fill(f.species_configs['forcing_scale'], df_scaling.loc[batch_start:batch_end-1, 'minorGHG'].values.squeeze(), specie=specie)
    fill(f.species_configs['forcing_scale'], 1 + 0.561*(calibrated_f4co2_mean - df_cr.loc[batch_start:batch_end-1,'F_4xCO2'].values)/calibrated_f4co2_mean, specie='CO2')

    # ozone
    for specie in df_ozone:
        fill(f.species_configs['ozone_radiative_efficiency'], df_ozone.loc[batch_start:batch_end-1, specie], specie=specie)
    
    # tune down volcanic efficacy
    fill(f.species_configs['forcing_efficacy'], 0.6, specie='Volcanic')
    
    # initial condition of CO2 concentration (but not baseline for forcing calculations)
    fill(f.species_configs['baseline_concentration'], df_1765co2.loc[batch_start:batch_end-1, 'co2_concentration'].values.squeeze(), specie='CO2')

    # initial condition of other species
    fill(f.species_configs['baseline_concentration'], baseline1765ch4, specie='CH4')
    fill(f.species_configs['baseline_concentration'], baseline1765n2o, specie='N2O')
    fill(f.species_configs['baseline_emissions'], 2, specie='Sulfur')
    fill(f.species_configs['baseline_emissions'], 174, specie='CO')
    fill(f.species_configs['baseline_emissions'], 10, specie='VOC')
    fill(f.species_configs['baseline_emissions'], 4, specie='NH3')
    fill(f.species_configs['baseline_emissions'], 2*46/14, specie='NOx')
    fill(f.species_configs['baseline_emissions'], 1.2, specie='BC')
    fill(f.species_configs['baseline_emissions'], 10, specie='OC')
    
    # initial conditions
    initialise(f.concentration, f.species_configs['baseline_concentration'])
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    initialise(f.cumulative_emissions, 0)
    initialise(f.airborne_emissions, 0)
    
    f.run(progress=False)
    
    # at this point dump out some batch output and put the constraining in another notebook
    temp_out[:, batch_start:batch_end] = f.temperature[85:, 0, :, 0]
    ohc_out[batch_start:batch_end] = f.ocean_heat_content_change[253, 0, :]-f.ocean_heat_content_change[206, 0, :]
    co2_out[batch_start:batch_end] = f.concentration[249, 0, :, 2]
    fari_out[batch_start:batch_end] = f.forcing[240:250, 0, :, 42].mean(axis=0)
    faci_out[batch_start:batch_end] = f.forcing[240:250, 0, :, 43].mean(axis=0)
    fo3_out[batch_start:batch_end] = f.forcing[254, 0, :, 44]
    ecs[batch_start:batch_end] = f.ebms.ecs
    tcr[batch_start:batch_end] = f.ebms.tcr

In [ ]:
pl.plot(f.temperature[:254, 0, :, 0]);

In [ ]:
pl.plot(f.concentration[:254, 0, :, 4]);

In [ ]:
pl.plot(f.forcing[:254, 0, :, 43]);

In [ ]:
pl.plot(f.forcing[:254, 0, :, 42]);

In [ ]:
pl.plot(f.forcing[:254, 0, :, 44]);

In [ ]:
pl.plot(f.concentration[:, 0, :, 3]);

In [ ]:
pl.plot(f.concentration[:, 0, :, 2]);

In [ ]:
np.save('../data/ar6_ensemble_batches_rcp/temperature_1850-2100.npy', temp_out, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/ohc_2018_minus_1971.npy', ohc_out, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/co2_2014.npy', co2_out, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/fari_2005-2014_mean.npy', fari_out, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/faci_2005-2014_mean.npy', faci_out, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/fo3_2019.npy', fo3_out, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/ecs.npy', ecs, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/tcr.npy', tcr, allow_pickle=True)